<a href="https://colab.research.google.com/github/ELiTE0005/DeepLearningTechniques/blob/main/week_11_12_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Week 11

In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

class VAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 400)
        self.fc_mu = nn.Linear(400, 20)
        self.fc_logvar = nn.Linear(400, 20)
        self.fc2 = nn.Linear(20, 400)
        self.fc3 = nn.Linear(400, 784)

    def encode(self, x):
        h = torch.relu(self.fc1(x))
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = (0.5 * logvar).exp()
        return mu + std * torch.randn_like(std)

    def decode(self, z):
        h = torch.relu(self.fc2(z))
        return torch.sigmoid(self.fc3(h))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar, z

def loss_fn(recon, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

train_loader = DataLoader(
    datasets.MNIST('.', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=64, shuffle=True
)

model = VAE()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

data, _ = next(iter(train_loader))
data = data.view(-1, 784)

optimizer.zero_grad()
recon, mu, logvar, z = model(data)
loss = loss_fn(recon, data, mu, logvar)
loss.backward()
optimizer.step()

print(f"Input batch shape: {data.shape}")
print(f"Sample mu: {mu[0, :4].tolist()}")
print(f"Loss: {loss.item():.4f}")


100%|██████████| 9.91M/9.91M [00:00<00:00, 134MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 40.1MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 64.3MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.91MB/s]


Input batch shape: torch.Size([64, 784])
Sample mu: [-0.06443769484758377, -0.02626766264438629, 0.04772452265024185, 0.06973494589328766]
Loss: 35269.0547


Week 12

In [2]:
# Minimal Deep Convolutional GAN (DCGAN) for color image generation
import torch, torch.nn as nn, torch.optim as optim, torchvision as tv

# 1. Load CIFAR-10 dataset (color images)
tf = tv.transforms.Compose([
    tv.transforms.Resize(32),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((.5,)*3, (.5,)*3)   # normalize to [-1,1]
])
dl = torch.utils.data.DataLoader(tv.datasets.CIFAR10('.', True, tf, download=True), 128, True)

# 2. Basic setup
nz = 100                              # latent vector (noise) size
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 3. Generator network (upsamples noise → fake color image)
G = nn.Sequential(
    nn.ConvTranspose2d(nz,256,4,1,0), nn.ReLU(True),
    nn.ConvTranspose2d(256,128,4,2,1), nn.ReLU(True),
    nn.ConvTranspose2d(128,64,4,2,1), nn.ReLU(True),
    nn.ConvTranspose2d(64,3,4,2,1), nn.Tanh()    # output 3-channel image
).to(device)

# 4. Discriminator network (downsamples image → real/fake)
D = nn.Sequential(
    nn.Conv2d(3,64,4,2,1), nn.LeakyReLU(.2,True),
    nn.Conv2d(64,128,4,2,1), nn.LeakyReLU(.2,True),
    nn.Conv2d(128,1,4,2,1), nn.Sigmoid()        # outputs probability
).to(device)

# 5. Loss and optimizers
optG = optim.Adam(G.parameters(), lr=2e-4, betas=(.5,.999))
optD = optim.Adam(D.parameters(), lr=2e-4, betas=(.5,.999))
loss = nn.BCELoss()   # binary cross entropy loss

# 6. Training loop (adversarial training)
for epoch in range(2):   # small epochs for demo
  for x,_ in dl:
    x = x.to(device); b = x.size(0)
    r, f = torch.ones(b,device=device), torch.zeros(b,device=device)

    # --- Train Discriminator ---
    D.zero_grad()
    l1 = loss(D(x).view(b,-1).mean(1), r)                # real images
    z = torch.randn(b,nz,1,1,device=device)
    fake = G(z)
    l2 = loss(D(fake.detach()).view(b,-1).mean(1), f)    # fake images
    (l1+l2).backward(); optD.step()

    # --- Train Generator ---
    G.zero_grad()
    l3 = loss(D(fake).view(b,-1).mean(1), r)             # wants D to think fake = real
    l3.backward(); optG.step()
  print(f"Epoch {epoch+1} complete")

# 7. Save sample generated images
tv.utils.save_image((G(torch.randn(16,nz,1,1,device=device)).cpu()+1)/2, "gen.png", nrow=4)
print("Generated images saved as gen.png")


100%|██████████| 170M/170M [00:02<00:00, 84.8MB/s]


Epoch 1 complete
Epoch 2 complete
Generated images saved as gen.png


Week 13

In [3]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load pre-trained ResNet18
model = models.resnet18(pretrained=True)

# Freeze convolutional layers
for param in model.parameters():
    param.requires_grad = False

# Replace the classification head for new task (5 classes)
model.fc = nn.Linear(model.fc.in_features, 5)

# Print only the new head and trainable layers
print("Modified classification head:", model.fc)
trainable = [name for name, p in model.named_parameters() if p.requires_grad]
print("Trainable layers:", trainable)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 153MB/s]


Modified classification head: Linear(in_features=512, out_features=5, bias=True)
Trainable layers: ['fc.weight', 'fc.bias']
